<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

In [ ]:
!unzip mallet-2.0.8.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python -m pip install --upgrade pip
!pip install pyLDAvis
# import pyLDAvis
import pyLDAvis.gensim_models # don't skip this

from pprint import pprint
import time
import pandas as pd
import re
import string
import nltk
from gensim.models import CoherenceModel
import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import collections

try:
    collectionsAbc = collections.abc
except AttributeError:
    collectionsAbc = collections 
try:
    from collections.abc import Callable  # noqa
except ImportError:
    from collections import Callable  # noqa



In [ ]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 1000

In [ ]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

bug_report = pd.read_csv(csv_file)
bug_report

In [ ]:
%%time

bug_report = bug_report.drop(columns=['Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
bug_report = bug_report.dropna(axis=0, subset=['Description'])
bug_report[bug_report['Description'].map(len) > 4]

# bug_report.head(1)

In [ ]:
%%time

bug_report["Description"] = bug_report["Description"].str.replace("Fixed in HEAD", "")
bug_report["Description"] = bug_report["Description"].str.replace("Has been marked as read-only", " ")

In [8]:
%%time

def cleanTextFromDescriptionAndTitle(report):

  report = report.replace('\t', '')
  report = re.sub('\w*\d\w*', '', report)
  report = report.lower()
  report = report.translate(str.maketrans('', '', string.punctuation))
    
  return report

bug_report['Description'] = bug_report['Description'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))
bug_report['Title'] = bug_report['Title'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))

# bug_report.loc[[0]]

CPU times: user 19.1 s, sys: 2.8 ms, total: 19.1 s
Wall time: 24.2 s


In [ ]:
result = bug_report.copy(deep=True)
result.reset_index(drop=True, inplace=True)
result.to_csv('drive/My Drive/result_reports.csv')
result

In [10]:
%%time

def wordTokenizationAndLemmatization(report):

    new_report = []
    for token in SIMPLE_PREPROCESS(report):
        if token not in STOP_WORDS:
          if len(token) > 5:
            new_report.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return new_report

bug_report['Title'] = bug_report['Title'].apply(lambda x: wordTokenizationAndLemmatization(x))
bug_report['Description'] = bug_report['Description'].apply(lambda x: wordTokenizationAndLemmatization(x))

# bug_report.head()

CPU times: user 30 s, sys: 744 ms, total: 30.7 s
Wall time: 31.7 s


In [11]:
%%time

duplicated_reports = bug_report.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True, inplace=True)
# duplicated_reports.to_csv('duplicate_reports.csv')
# duplicated_reports

CPU times: user 17 ms, sys: 1 ms, total: 18 ms
Wall time: 45.1 ms


In [12]:
%%time

master_reports = bug_report[bug_report.isnull().any(axis=1)]
master_reports.reset_index(drop=True, inplace=True)
# master_reports.to_csv('master_reports.csv')
# master_reports

CPU times: user 62.6 ms, sys: 0 ns, total: 62.6 ms
Wall time: 67.2 ms


In [ ]:
%%time

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for key, value in dictionary.iteritems():
  print(key, value)
  count += 1
  if count > 100:
      break


In [ ]:
%%time

bag_of_words_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in master_reports['Description']]
bag_of_words_corpus[0]

In [ ]:
%%time

bag_of_words_doc = bag_of_words_corpus[0]

for i in range(len(bag_of_words_doc)):
    print("{}. \"{}\" word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

In [ ]:
mallet_path = '/content/mallet-2.0.8/bin/mallet'

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=bag_of_words_corpus, num_topics=10, id2word=dictionary)

In [ ]:
import pickle
pickle.dump(ldamallet, open("drive/My Drive/ldamallet.pkl", "wb"))

In [14]:
ldamallet = pickle.load(open("drive/My Drive/ldamallet.pkl", "rb"))
# ldamallet.print_topics()

In [ ]:
from pprint import pprint
# display topics
pprint(ldamallet.show_topics(formatted=False))

In [ ]:
%%time


for c in range(10):
    exec('topic{} = pd.DataFrame()'.format(c))
    for i in range(len(master_reports)):
        topic=ldamallet[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic{} = topic{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic{} = topic{}.reset_index(drop=True)'.format(c,c))
            exec('topic{}.to_csv("topic{}.csv")'.format(c,c))

In [ ]:
%%time

for c in range(10):
  exec('topic{} = pd.read_csv("topic{}.csv", error_bad_lines=False, engine="python")'.format(c,c))
  exec("topic{}= topic{}.drop(columns=['Unnamed: 0'])".format(c,c))
  exec("topic{}['Description'] = topic{}['Description'].map(wordTokenizationAndLemmatization)".format(c,c))

In [16]:
%%time

for i in range(10):
  exec('sent{} = []'.format(i))
  exec('x = topic{}'.format(i))
  for j in range(len(x)):
    exec('sent{}.append(topic{}.Description[{}])'.format(i,i,j))

CPU times: user 1.78 s, sys: 5.43 ms, total: 1.79 s
Wall time: 1.81 s


In [ ]:
%%time

#Length of all the cluster
for sent in range(10):
  exec('print(len(sent{}))'.format(sent))
  

In [ ]:
%%time

corpus = bag_of_words_corpus

lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

pprint(lda_model.print_topics())

In [25]:
file = open('drive/My Drive/dictionary.pickle', 'wb')

# dump information to that file
pickle.dump(dictionary, file)

In [26]:
f= open('drive/My Drive/bow_corpus.pickle', 'wb')

# dump information to that file
pickle.dump(bag_of_words_corpus, f)

In [27]:
f = open('drive/My Drive/bow_corpus.pickle', 'rb')
bow_corpus=pickle.load(f)

file = open('drive/My Drive/dictionary.pickle', 'rb')
dictionary=pickle.load(file)



EOFError: ignored

In [ ]:
# later on, load trained model from file
lda_model =  models.LdaModel.load('drive/My Drive/lda_model.model')
pprint(lda_model.print_topics())

In [ ]:
lda_model.save('drive/My Drive/lda_model.model')

In [ ]:
%%time

print('\nPerplexity: ', lda_model.log_perplexity(bag_of_words_corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=bug_report['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
%%time

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bag_of_words_corpus, dictionary)
vis

In [ ]:
unseen_document = "setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with."

topic=wordTokenizationAndLemmatization(unseen_document)

# bow_vector = topic=ldamallet[dictionary.doc2bow(wordTokenizationAndLemmatization(unseen_document))]
print(topic)

# for index, score in sorted(ldamallet[bow_vector], key=lambda tup: -1*tup[1]):
#     print("Score: {}\t Topic: {}".format(score, ldamallet.print_topic(index, 10)))

['project', 'contain', 'resource', 'release', 'project', 'resource', 'external', 'editor', 'paintshop', 'external', 'editor', 'navigator', 'resource', 'verify', 'change', 'release', 'project', 'release', 'navigator', 'resource', 'verify', 'change', 'problem', 'refresh', 'workspace', 'change', 'release', 'open', 'resource']


In [ ]:
%%time


for c in range(10):
    exec('topic_{} = pd.DataFrame()'.format(c))
    for i in range(len(master_reports)):
        topic=lda_model[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(c,c))
            exec('topic_{}.to_csv("topic_{}.csv")'.format(c,c))


In [ ]:
%%time

for c in range(10):
  exec('topic_{} = pd.read_csv("topic_{}.csv", error_bad_lines=False, engine="python")'.format(c,c))
  exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
  exec("topic_{}['Description'] = topic_{}['Description'].map(wordTokenizationAndLemmatization)".format(c,c))

CPU times: user 10.7 s, sys: 295 ms, total: 11 s
Wall time: 11 s


In [ ]:
%%time

for i in range(10):
  exec('sent_{} = []'.format(i))
  exec('x = topic_{}'.format(i))
  for j in range(len(x)):
    exec('sent_{}.append(topic_{}.Description[{}])'.format(i,i,j))

CPU times: user 1.72 s, sys: 11.7 ms, total: 1.73 s
Wall time: 1.75 s


In [ ]:
%%time

#Length of all the cluster
for sent in range(10):
  exec('print(len(sent_{}))'.format(sent))
  

In [ ]:
!pip install glove-python-binary

In [32]:
from gensim.models import Word2Vec,FastText
from gensim.test.utils import get_tmpfile
from glove import Glove, Corpus
import glove

In [ ]:
%%time

for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent{}'.format(cluster))    # Path to a corpus file
  vector_size = 100                           # Dimensionality of the word vectors
  w = 6                                       #Maximum distance between the current and predicted word within a sentence
  min_count = 5                               # Ignores all words with total frequency lower than this.
  CBoW = 0                                    # Training algorithm: 1 for skip-gram; otherwise CBOW(The model tries to predict the target word by trying to understand the context of the surrounding words.).
  iterations = 100

  exec('w2vmodel_ldamallet{} = Word2Vec(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  exec('path = get_tmpfile("drive/My Drive/word2vec_ldamallet{}.model")'.format(cluster))
  exec('w2vmodel_ldamallet{}.save("drive/My Drive/word2vec_ldamallet{}.model")'.format(cluster, cluster))

CPU times: user 4min 49s, sys: 2.55 s, total: 4min 51s
Wall time: 2min 46s


In [ ]:
%%time

for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent{}'.format(cluster))
  vector_size = 100
  w = 6
  min_count = 5
  CBoW = 0
  iterations = 100

  #Training FastText model for each cluster
  exec ('ftmodel_ldamallet{} = FastText(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("drive/My Drive/ftmodel_ldamallet{}.model")'.format(cluster))
  exec('ftmodel_ldamallet{}.save("drive/My Drive/ftmodel_ldamallet{}.model")'.format(cluster, cluster))

CPU times: user 31min 57s, sys: 7.77 s, total: 32min 5s
Wall time: 17min 17s


In [ ]:
%%time

for cluster in range(10):
  vector_size = 100
  exec('glove_corpus{}=Corpus()'.format(cluster, cluster)) 
  exec('glove_corpus{}.fit(sent{})'.format(cluster, cluster))
  exec('glove_ldamallet{}= Glove(no_components=vector_size, learning_rate=0.18, alpha=0.75, max_count=100, max_loss=10.0, random_state=None)'.format(cluster, cluster))
  exec('glove_ldamallet{}.fit(glove_corpus{}.matrix, epochs=1000, no_threads=3, verbose=True)'.format(cluster, cluster))
  exec('transformer = lambda dictionary2:glove_m{}.transform_paragraph(words, epochs=1000,ignore_missing=False)'.format(cluster, cluster))
  exec('glove_ldamallet{}.add_dictionary(glove_corpus{}.dictionary)'.format(cluster, cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("drive/My Drive/glove_ldamallet{}.model")'.format(cluster))
  exec('glove_ldamallet{}.save("drive/My Drive/glove_ldamallet{}.model")'.format(cluster, cluster))

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from scipy.spatial import distance
warnings.filterwarnings("ignore", category=DeprecationWarning)
from glove import Glove, Corpus

In [47]:
test = duplicated_reports
test.head()
print(len(test))

14398


In [33]:
for mod in range(10):
  #import all the trained Word2Vec models
  exec('word2vec_ldamallet{} = Word2Vec.load("drive/My Drive/word2vec_ldamallet{}.model")'.format(mod, mod))

  #import all the trained FastText models
  exec('ftmodel_ldamallet{} = FastText.load("drive/My Drive/ftmodel_ldamallet{}.model")'.format(mod, mod))

  #import all the trained GloVe models
  exec('glove_ldamallet{} = Glove.load("drive/My Drive/glove_ldamallet{}.model")'.format(mod, mod))

In [35]:
for mod in range(10):
    exec("print(list(word2vec_ldamallet{}.wv.vocab))".format(mod))

['random', 'comment', 'package', 'method', 'rename', 'shouldnt', 'interface', 'plugin', 'implementations', 'public', 'cvsfile', 'remove', 'folder', 'special', 'status', 'delete', 'follow', 'methods', 'getname', 'replace', 'getlabel', 'return', 'repository', 'display', 'propagate', 'exception', 'appropriate', 'message', 'duplicate', 'subclass', 'settitle', 'createcontents', 'iresource', 'notification', 'debugger', 'variables', 'change', 'different', 'control', 'present', 'treeitems', 'example', 'static', 'string', 'treeshell', 'swtnone', 'treesetbounds', 'treeitem', 'treeitemtree', 'itemsettextitem', 'canvas', 'accent', 'fontdata', 'fontdisplay', 'listener', 'treeeditor', 'editor', 'treeeditortree', 'swtleft', 'gctextextent', 'gcdispose', 'shellopen', 'shellisdisposed', 'displaysleep', 'fontdispose', 'displaydispose', 'language', 'variable', 'presentation', 'specific', 'support', 'options', 'extend', 'treeviewer', 'fairly', 'behavior', 'outline', 'individual', 'easier', 'feature', 'requ

In [36]:
def sim_with_clusters_lda_topn(DR, n):
    vec_bow = dictionary.doc2bow(DR)
    x= ldamallet[vec_bow]
    topic = np.asarray(x)
    # max_sim = int(topic[np.argmax(topic[:,1]),0]) 
    # max_sim
    sim=[]
    x= topic[np.argsort(topic[:,1])[-n:][::-1],0]
    for i in range(len(x)):
        sim.append(int(x[i]))
    # return max_sim
    return sim


In [37]:
def average_word_vectors_glove(words, model, vocabulary, num_features):  
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.  

    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.word_vectors[model.dictionary[word]])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    


def averaged_word_vectorizer_glove(corpus, model, num_features):
    vocabulary = set(model.dictionary)
    if(any(isinstance(i, list) for i in corpus)):
      features = [average_word_vectors_glove(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
      return np.array(features)
    else:
      features = average_word_vectors_glove(corpus, model, vocabulary, num_features)
      return np.array(features)


In [38]:
def average_word_vectors_ft(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    
def averaged_word_vectorizer_ft(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    if(any(isinstance(i, list) for i in corpus)):
      features = [average_word_vectors_ft(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
      return np.array(features)
    else:
      features = average_word_vectors_ft(corpus, model, vocabulary, num_features)
      return np.array(features)

In [39]:
def average_word_vectors_word2vec(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector

def averaged_word_vectorizer_word2vec(corpus, model, num_features):
    vocabulary = set(model.wv.vocab)
    if(any(isinstance(i, list) for i in corpus)):
      features = [average_word_vectors_word2vec(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
      return np.array(features)
    else:
      features = average_word_vectors_word2vec(corpus, model, vocabulary, num_features)
      return np.array(features)

In [40]:
def sim(vec1, vec2): 
  sim1 = 1/(1+distance.euclidean(vec1, vec2))
  sim2 = cosine_similarity(vec1, vec2)
  sim=(sim1+sim2)/2 
  return sim

In [41]:
def fusion3(vec1, vec2, vec3, vec4, vec5, vec6, fusion_no):
    if (fusion_no == '4'):
        master = np.concatenate((vec1, vec2, vec3), axis=1)
        pca = PCA(n_components=100)
        # avg_fit = pca.fit(master)
        master = pca.fit_transform(master)
        vec_duplicate = np.concatenate((vec4, vec5, vec6), axis=0)
        vec_duplicate=[vec_duplicate]
        vec_duplicate = pca.transform(vec_duplicate)
        return vec_duplicate, master

    else:
        raise ValueError('Invalid value for fusion')

In [42]:
def fusion2(vec1, vec2, vec3, vec4, fusion_no):
    if (fusion_no == '2'):
        master = np.concatenate((vec1, vec2), axis=1)
        pca = PCA(n_components=50)
        avg_fit = pca.fit(master)
        master = pca.transform(master)
        vec_duplicate = np.concatenate((vec3, vec4), axis=0)
        vec_duplicate=[vec_duplicate]
        vec_duplicate = pca.transform(vec_duplicate)
        return vec_duplicate, master
    
    else:
        raise ValueError('Invalid value for fusion')

In [43]:
# creation of feature vectors by multimodality feature extraction
def feature_vectors_multi_modality(DR, corpus, model1, model2, model3, fusion_no):

    master_ft1 = averaged_word_vectorizer_ft(corpus=sent, model=model1, num_features=100)
    master_glove2 = averaged_word_vectorizer_glove(corpus=sent, model=model2, num_features=100)
    master_w2v3 = averaged_word_vectorizer_word2vec(corpus=sent, model=model3, num_features=100)

    vec_duplicate1 = averaged_word_vectorizer_ft(corpus=DR, model=model1, num_features=100)
    vec_duplicate2 = averaged_word_vectorizer_glove(corpus=DR, model=model2, num_features=100)
    vec_duplicate3 = averaged_word_vectorizer_word2vec(corpus=DR, model=model3, num_features=100)

    vec_duplicate , master= fusion3(master_ft1,  master_glove2, master_w2v3, vec_duplicate1,  vec_duplicate2, vec_duplicate3, fusion_no)
    

    return vec_duplicate,master

In [44]:
def feature_vectors_single_modality(DR, corpus, model1):
  master = averaged_word_vectorizer_ft(corpus=sent, model=model1, num_features=100)

  vec_duplicate = averaged_word_vectorizer_ft(corpus=DR, model=model1, num_features=100)

  vec_duplicate = [vec_duplicate]

  return vec_duplicate, master

In [45]:
def compare_topn(model1, model2, model3, cluster, sent, DR, topn, modal, fusion_no):
    similarity=[]

    if (modal == 'multi'):
        vec_duplicate, master= feature_vectors_multi_modality(DR, sent, model1, model2, model3, fusion_no)
    elif (modal == 'single'):
        vec_duplicate, master= feature_vectors_single_modality(DR, sent, model1)
    else:
        raise ValueError('Invalid Modality entered')

    for doc in range(len(master)):
        vec_master = master[doc]
        vec_master= [vec_master]
        unified_sim = sim(vec_duplicate, vec_master)

        similarity.append(unified_sim)

    similarity = np.asarray(similarity)
    similarity= np.concatenate(similarity, axis=0 )
    similarity= np.concatenate(similarity, axis=0 )
    max_similar_reports=similarity.argsort()[-topn:][::-1]

    return(max_similar_reports)

In [ ]:
vec_acc=[]
t1 = time.time()
sum = 0
no_of_test_samples= int(200)
for i in range(200):
  sample = test.Description[i] 
  n = 1
  max_cluster = sim_with_clusters_lda_topn(sample, n)
  v=[]
  for max in max_cluster:
    exec('cluster = topic{}'.format(max))              
    exec('model1 = ftmodel_ldamallet{}'.format(max))              
    exec('model2 = glove_ldamallet{}'.format(max))  
    exec('model3 = word2vec_ldamallet{}'.format(max))              
    exec('sent = topic{}.Description'.format(max))     
    topn = 833                                          
    fusion_no = '4'   
    
    modal = 'multi'                                    
     
    max_sim = compare_topn(model1, model2, model3, cluster, sent, sample, topn, modal, fusion_no)
    t2 = time.time()

    
    for num in max_sim:
      if (cluster.Issue_id[num] == test.Duplicated_issue[i]):
          v.append("1")
      else:
          v.append("0")
  
  if(all(x==v[0] for x in v)):
    vec_acc.append("0")
  else:
    vec_acc.append("1")



for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))

In [55]:

sample = 'Online Grammar and Writing Checker To Help You Deliver Impeccable, Mistake-free Writing. Grammarly Has a Tool For Just About Every Kind Of Writing You Do. Try It Out For Yourself! Easily Improve Any Text. Find and Add Sources Fast. Eliminate Grammar Errors.'
sample = wordTokenizationAndLemmatization(sample)

max_cluster = sim_with_clusters_lda_topn(sample, 1)
print(max_cluster)

for max in max_cluster:
    exec('cluster = topic{}'.format(max))              
    exec('model1 = ftmodel_ldamallet{}'.format(max))              
    exec('model2 = glove_ldamallet{}'.format(max))  
    exec('model3 = word2vec_ldamallet{}'.format(max))             
    exec('sent = topic{}.Description'.format(max))     
    topn = 10                                          
    fusion_no = '4'   
    modal = 'multi'
    max_sim = compare_topn(model1, model2, model3, cluster, sent, sample, topn, modal, fusion_no)
    max_sim = max_sim.tolist()
    print(max_sim)
    
    issue_id = []
    for num in max_sim:
        id = cluster.Issue_id[num]
        issue_id.append(id)

    #   title = (result.loc[result['Issue_id'] == id, 'Title']).to_string(index = False)
    #   desc = (result.loc[result['Issue_id'] == id, 'Description']).to_string(index = False)
    #   priority = (result.loc[result['Issue_id'] == id, 'Priority']).to_string(index = False)
    #   team = (result.loc[result['Issue_id'] == id, 'Component']).to_string(index = False)

    #   print('Title: ', title)
    #   print('Description: ', desc)
    #   print('Priority:', priority)
    #   print('Team:', team)
    print(issue_id)
    
   

[3]
[1936, 11048, 10472, 10598, 11126, 10920, 5311, 10055, 2135, 1795]
[30817, 400753, 343531, 362844, 407707, 389048, 93284, 302436, 34994, 28896]
